<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno: José Luis Cádiz Sejas



### **Link de repositorio de GitHub:** `https://github.com/JLCdata/Programacion_Data_Science`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Exploración del dataset y los resultados a través de visualizaciones interactivas usando `plotly`.
- Introducción a procesamiento del lenguaje natural.
- Clasificación en `scikit-learn`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 26/11/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Aplicar las ventajas que nos ofrece crear un pipeline.
- Obtener caracteristicas desde texto.
- Visualizar el funcionamiento de clasificadores.
- Realizar una GridSearch sobre un conjunto de clasificadores.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
#!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to C:\Users\Jose
[nltk_data]     Luis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [2]:
df_comics = pd.read_csv('df_comics.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia
df_comics_no_label = pd.read_csv('comics_no_label.csv')

In [3]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.drop(["Unnamed: 0"],axis=1,inplace=True)
df_comics.head()

name               real_name               full_name overall_score  \
0       3-D Man     Delroy Garrett, Jr.     Delroy Garrett, Jr.             6   
1        A-Bomb  Richard Milhouse Jones  Richard Milhouse Jones            20   
2            Aa                      Aa                     NaN            12   
3    Aaron Cash              Aaron Cash              Aaron Cash             5   
4  Aayla Secura            Aayla Secura                     NaN             8   

                                        history_text  \
0  Delroy Garrett, Jr. grew up to become a track ...   
1   Richard "Rick" Jones was orphaned at a young ...   
2  Aa is one of the more passive members of the P...   
3  Aaron Cash is the head of security at Arkham A...   
4  ayla Secura was a Rutian Twi'lek Jedi Knight (...   

                                         powers_text  intelligence_score  \
0                                                NaN                  85   
1    On rare occasions, and through unusual circu...                  80   
2                                                NaN                  80   
3                                                NaN                  80   
4                                                NaN                  90   

   strength_score  speed_score  durability_score  power_score  combat_score  \
0              30           60                60           40            70   
1             100           80               100          100            80   
2              50           55                45          100            55   
3              10           25                40           30            50   
4              40           45                55           55            85   

                                         superpowers alter_egos  \
0                  ['Super Speed', 'Super Strength']         []   
1  ['Accelerated Healing', 'Agility', 'Berserk Mo...         []   
2  ['Energy Absorption', 'Energy Armor', 'Energy ...         []   
3          ['Weapon-based Powers', 'Weapons Master']         []   
4  ['Accelerated Healing', 'Agility', 'Astral Pro...         []   

          aliases      place_of_birth  \
0            ['']                 NaN   
1  ['Rick Jones']  Scarsdale, Arizona   
2            ['']          Stoneworld   
3            ['']         Gotham City   
4            ['']                 NaN   

                          first_appearance        creator alignment  \
0                                      NaN  Marvel Comics      Good   
1  Hulk Vol 2 #2 (April, 2008) (as A-Bomb)  Marvel Comics      Good   
2                  Green Lantern Vol 3 #21      DC Comics      Good   
3                                      NaN      DC Comics      Good   
4                                      NaN   George Lucas      Good   

                                          occupation base  \
0                                                NaN  NaN   
1  Musician, adventurer, author; formerly talk sh...  NaN   
2                                                NaN  NaN   
3                                                NaN  NaN   
4                                                NaN  NaN   

                                               teams  \
0  ['Annihilators', 'Asgardians', 'Avengers', 'Ne...   
1  ['Teen Brigade', 'Ultimate Fantastic Four', 'U...   
2  ['Blue Lantern Corps', 'Green Lantern Corps', ...   
3                                                 []   
4                                     ['Jedi Order']   

                                           relatives  gender type_race  \
0                                                NaN    Male     Human   
1  Marlo Chandler-Jones (wife); Polly (aunt); Mrs...    Male     Human   
2                                                NaN    Male     Human   
3                                                NaN    Male     Human   
4                                                NaN  Female   Twi'lek   

         height           wei

## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `bag of words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `bag of words`?...

`bag of words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `bag of words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `bag of words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [4]:
import nltk
#nltk.download()
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [5]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to C:\Users\Jose
[nltk_data]     Luis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'beauti', 'citi']]

In [6]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [7]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

&  actor  beauti  best  citi  good  in  is  like  new  rock  roll  teacher  \
0  1      0       0     0     0     1   0   0     1    0     2     1        1   
1  0      1       0     1     0     0   1   1     0    0     1     0        0   
2  0      0       1     0     1     0   0   1     0    1     0     0        0   

   the  world  york  
0    1      0     0  
1    3      1     0  
2    0      0     1

Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [8]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

&  & roll  actor  actor in  beauti  beauti citi  best  best actor  citi  \
0  1       1      0         0       0            0     0           0     0   
1  0       0      1         1       0            0     1           1     0   
2  0       0      0         0       1            1     0           0     1   

   good  good rock  in  in the  is  is beauti  is the  like  like good  new  \
0     1          1   0       0   0          0       0     1          1    0   
1     0          0   1       1   1          0       1     0          0    0   
2     0          0   0       0   1          1       0     0          0    1   

   new york  rock  rock &  rock is  rock like  roll  teacher  teacher rock  \
0         0     2       1        0          1     1        1             1   
1         0     1       0        1          0     0        0             0   
2         1     0       0        0          0     0        0             0   

   the  the best  the rock  the teacher  the world  world  york  york is  
0    1         0         0            1          0      0     0        0  
1    3         1         1            0          1      1     0        0  
2    0         0         0            0          0      0     1        1

De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [x] Obtener a traves de bag of words caracteristicas del resumen de historia de cada personaje.
- [x] Aplicar MinMaxScaler sobre los atributos de interes.

**Respuesta:**

In [11]:
# Variables categoricas (string)
categorical_features="history_text"

# Variables númericas
atributos_de_interes = ['intelligence_score',
                        'strength_score', 
                        'speed_score',
                        'durability_score',
                        'power_score',
                        'combat_score']

In [33]:
# Columns transformer
preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('bow',  CountVectorizer(tokenizer= StemmerTokenizer(),
                      ngram_range=(1,2) 
                      ),  
          categorical_features),
        ('MinMaxScaler',MinMaxScaler(),
         atributos_de_interes),
    ])

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>


Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Finalmente entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

**To-DO:**
- [x] Realizar un pipeline con las caracteristicas solicitadas en 1.1 y aplicar un clasificador  `MultinomialNB()`.
- [x] Entrenar el pipeline y comentar los resultados.

**Respuesta:**

In [13]:
# Pipeline

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
     ("Selection", SelectPercentile(f_classif, percentile=90)),
     ("Bayes",MultinomialNB())
    ]
)

In [14]:
# Generación conjunto de train y test

features = df_comics.drop(columns=["alignment"])[["history_text",
                        'intelligence_score',
                        'strength_score', 
                        'speed_score',
                        'durability_score',
                        'power_score',
                        'combat_score']].copy() 

labels = df_comics.loc[:, "alignment"].copy()

# Generación de conjuntos de train y test
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.33, 
    shuffle=True,
    stratify=labels
)

In [15]:
df_comics.loc[:, "alignment"].value_counts()

Good       743
Bad        429
Neutral    113
Name: alignment, dtype: int64

In [16]:
# Entrenamiento de pipeline
pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('bag of words',
                                                  CountVectorizer(ngram_range=(1,
                                                                               2),
                                                                  tokenizer=<__main__.StemmerTokenizer object at 0x000001C5ED504970>),
                                                  'history_text'),
                                                 ('MinMaxScaler',
                                                  MinMaxScaler(),
                                                  ['intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                                   'power_score',
                                                   'c

In [17]:
# Generación de predicciones
y_pred = pipe.predict(X_test)

In [18]:
# Métricas de evaluación 
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

         Bad       0.06      0.67      0.10        12
        Good       0.99      0.59      0.74       413
     Neutral       0.00      0.00      0.00         0

    accuracy                           0.59       425
   macro avg       0.35      0.42      0.28       425
weighted avg       0.96      0.59      0.72       425



**Dado el bajo precision para la clase Bad, por lo que hay baja certeza sobre la veracidad de sus predicciones. Debido a esto no se recomienda usar este clasificador Baseline.**

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Dos clasificadores distintos en donde varie sus parámetros. Se le recomienda utilizar `LogisticRegression()` y `RandomForestClassifier()`.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. 
- Selección de las mejores columnas para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
        'clf__multi_class': ['liblinear']},
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [77]:
pipe1 = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
     ("Selection", SelectPercentile(f_classif, percentile=90)),
     ("clf",LogisticRegression(random_state=1,C=0.1))
    ]
)

In [78]:
param_grid = [{'Selection__percentile': [20, 40, 60, 80]},
              {'preprocesamiento__bow__ngram_range': [(1,1), (1,2),(1,3)]},
       # clasificador 1 + hiperparámetros
       {'clf':[LogisticRegression(random_state=1)],
        'clf__C': [0.01,1000]},
       # clasificador 2 + hiperparámetros    
       {'clf': [RandomForestClassifier(random_state=1)],
        'clf__n_estimators': [200,1000]}
      
       ]

In [61]:
'''
gs = GridSearchCV(pipe1, param_grid,verbose=10)
gs.fit(X_train, y_train)
gs.best_params_
gs.best_estimator_.get_params()
pd.DataFrame(gs.cv_results_)
# Generación de predicciones
y_pred = gs.predict(X_test)
# Métricas de evaluación 
print(classification_report(y_pred, y_test))
'''

'\ngs = GridSearchCV(pipe1, param_grid,verbose=10)\ngs.fit(X_train, y_train)\ngs.best_params_\ngs.best_estimator_.get_params()\npd.DataFrame(gs.cv_results_)\n# Generación de predicciones\ny_pred = gs.predict(X_test)\n# Métricas de evaluación \nprint(classification_report(y_pred, y_test))\n'

In [79]:
# Halving Grid Search
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingGridSearchCV

hgs = HalvingGridSearchCV(pipe1, param_grid, verbose=10)


In [80]:
# Fit
hgs.fit(X_train, y_train)

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 95
max_resources_: 860
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 11
n_resources: 95
Fitting 5 folds for each of 11 candidates, totalling 55 fits
[CV 1/5; 1/11] START Selection__percentile=20...................................
[CV 1/5; 1/11] END Selection__percentile=20;, score=(train=0.987, test=0.526) total time=   1.7s
[CV 2/5; 1/11] START Selection__percentile=20...................................
[CV 2/5; 1/11] END Selection__percentile=20;, score=(train=0.987, test=0.632) total time=   3.3s
[CV 3/5; 1/11] START Selection__percentile=20...................................
[CV 3/5; 1/11] END Selection__percentile=20;, score=(train=1.000, test=0.632) total time=   2.3s
[CV 4/5; 1/11] START Selection__percentile=20...................................
[CV 4/5; 1/11] END Selection__percentile=20;, score=(train=0.987, test=0.474) total time=   3.1s
[CV 5/5; 1/11] START Selection

[CV 3/5; 9/11] END clf=LogisticRegression(random_state=1), clf__C=1000;, score=(train=1.000, test=0.684) total time=   3.4s
[CV 4/5; 9/11] START clf=LogisticRegression(random_state=1), clf__C=1000........
[CV 4/5; 9/11] END clf=LogisticRegression(random_state=1), clf__C=1000;, score=(train=1.000, test=0.579) total time=   3.9s
[CV 5/5; 9/11] START clf=LogisticRegression(random_state=1), clf__C=1000........
[CV 5/5; 9/11] END clf=LogisticRegression(random_state=1), clf__C=1000;, score=(train=1.000, test=0.421) total time=   3.8s
[CV 1/5; 10/11] START clf=RandomForestClassifier(random_state=1), clf__n_estimators=200
[CV 1/5; 10/11] END clf=RandomForestClassifier(random_state=1), clf__n_estimators=200;, score=(train=1.000, test=0.526) total time=   1.9s
[CV 2/5; 10/11] START clf=RandomForestClassifier(random_state=1), clf__n_estimators=200
[CV 2/5; 10/11] END clf=RandomForestClassifier(random_state=1), clf__n_estimators=200;, score=(train=1.000, test=0.684) total time=   3.2s
[CV 3/5; 10/

[CV 3/5; 2/2] END preprocesamiento__bow__ngram_range=(1, 2);, score=(train=0.999, test=0.678) total time=  31.1s
[CV 4/5; 2/2] START preprocesamiento__bow__ngram_range=(1, 2)...................
[CV 4/5; 2/2] END preprocesamiento__bow__ngram_range=(1, 2);, score=(train=0.999, test=0.608) total time=  29.9s
[CV 5/5; 2/2] START preprocesamiento__bow__ngram_range=(1, 2)...................
[CV 5/5; 2/2] END preprocesamiento__bow__ngram_range=(1, 2);, score=(train=0.999, test=0.719) total time=  31.7s


HalvingGridSearchCV(estimator=Pipeline(steps=[('preprocesamiento',
                                               ColumnTransformer(transformers=[('bow',
                                                                                CountVectorizer(ngram_range=(1,
                                                                                                             2),
                                                                                                tokenizer=<__main__.StemmerTokenizer object at 0x000001C5ED541EB0>),
                                                                                'history_text'),
                                                                               ('MinMaxScaler',
                                                                                MinMaxScaler(),
                                                                                ['intelligence_score',
                                                                

In [81]:
# Predict & Eval
y_pred = hgs.predict(X_test)
# Métricas de evaluación 
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

         Bad       0.58      0.65      0.61       128
        Good       0.87      0.73      0.79       292
     Neutral       0.08      0.60      0.14         5

    accuracy                           0.70       425
   macro avg       0.51      0.66      0.52       425
weighted avg       0.77      0.70      0.73       425



**Ahora se obtuvieron mejores resultados, por lo que se procede a evaluar la vaca.**

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [97]:
xnew[xnew["name"].isin(["Vergil","Gorilla Girl","Batcow"])].drop_duplicates().drop(["name"],axis=1)

history_text  intelligence_score  \
16  Bat-Cow was originally a cow that was found by...                  70   
40  A carnival performer with the ability to turn ...                  90   
78  Vergil, later also known as Nelo Angelo, is on...                  90   

    strength_score  speed_score  durability_score  power_score  combat_score  
16              10           25                20           10            20  
40              35           60                60           45           100  
78              75           95                90          100           100

In [91]:
# Data a predecir
xnew=df_comics_no_label[["name","history_text",
                        'intelligence_score',
                        'strength_score', 
                        'speed_score',
                        'durability_score',
                        'power_score',
                        'combat_score']]
xnew= xnew.dropna(subset=['history_text'])
xnew.head()

name                                       history_text  \
0     514A (Gotham)  He was one of the many prisoners of Indian Hil...   
2             A'dal  As with most of the naaru, little is known of ...   
3  Agent Zero (FOX)  During mid-late 1973, Zero was a member of Tea...   
4        Ajax (FOX)  Ajax (born Francis Freeman) was a human who ga...   
6   A.M.A.Z.O. (CW)  The Anti Meta-human Adaptive Zootomic Organism...   

   intelligence_score  strength_score  speed_score  durability_score  \
0                 100              20           30                50   
2                  85              30           70                55   
3                  90              10           25                25   
4                  85              25           45                75   
6                  90             100          100               100   

   power_score  combat_score  
0           35           100  
2           65           100  
3           35            80  
4           45            65  
6          100           100

In [98]:
importantes=xnew[xnew["name"].isin(["Vergil","Gorilla Girl","Batcow"])].drop_duplicates().drop(["name"],axis=1)

In [104]:
# Predict
importantes["Predict"]=hgs.predict(importantes)
importantes

history_text  intelligence_score  \
16  Bat-Cow was originally a cow that was found by...                  70   
40  A carnival performer with the ability to turn ...                  90   
78  Vergil, later also known as Nelo Angelo, is on...                  90   

    strength_score  speed_score  durability_score  power_score  combat_score  \
16              10           25                20           10            20   
40              35           60                60           45           100   
78              75           95                90          100           100   

   Predict  
16     Bad  
40    Good  
78    Good

In [103]:
importantes["history_text"].iloc[0]

'Bat-Cow was originally a cow that was found by Robin during an investigation on a slaughterhouse, which the cow was sent to. Robin wanted to keep the cow and named it Bat-Cow. Batman decided they would keep the cow. During a later investigation, Batman discovered it had mind-controlling radiation inside of it, leading him to attempt to get rid of the cow. Robin, however would not allow him to hurt the cow. After the events of Robin\'s death, Bat-Cow and Robin\'s dog felt a great loss in morale. Alfred Pennyworth and Nightwing had to take care of the cow. Later after Nightwing\'s disappearance, Alfred was the only caretaker of Bat-Cow. Bat-Cow, complete with cape, spent time on a Wayne Enterprises-owned dairy farm. It was there the cow encountered the cosmic-powered "Forever People." One of them seemed to gain guidance from Bat-Cow\'s actions, namely when Bat-Cow gave one of the Forever People a scarecrow\'s hat. Bat-Cow formed a friendship with Titus, a dog and household cat.'

**Se concluye que en este caso la predicción de la vaca es correcta, dado que es radioactiva y contrala la mente de las personas.**

**Finalmente se concluye que si bien Halving reduce tiempos para terminar el grid, este no siempre encuentra una solución suficientemente buena como la obtenida con un GridSearch directamente. Para obtener los resultados se tuvo que iterar muchas veces sobre el Halving, por lo que de todas formas se perdio tiempo iterando para llegar a resultados similares con el GridSearch estandar.**

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>